In [ ]:
# Cell 1: Imports
import warnings
warnings.filterwarnings('ignore')

import subprocess
subprocess.run([sys.executable, "-m", "pip", "install", 
    "numpy", "pandas", "matplotlib", "seaborn","joblib", "scipy", "hopsworks"])

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hopsworks
import joblib
import shap
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

shap.initjs()
sns.set_theme(style="whitegrid", palette="muted")

HOPSWORKS_HOST = "eu-west.cloud.hopsworks.ai"
HOPSWORKS_PROJECT = "Islamabad_Aqi_Predictor"
ARTIFACT_DIR = Path("../artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

FEATURE_COLS = [
    "pm10", "carbon_monoxide", "nitrogen_dioxide",
    "ozone", "sulphur_dioxide",
    "hour", "day", "month",
    "pm2_5_change", "pm10_change",
]

print("Imports complete ✓")

ModuleNotFoundError: No module named 'joblib'

In [ ]:
# Cell 2: Connect to Hopsworks and load model
project = hopsworks.login(
    host=HOPSWORKS_HOST,
    project=HOPSWORKS_PROJECT,
    api_key_value=os.getenv("HOPSWORKS_API_KEY"),
)

mr = project.get_model_registry()
hw_model = mr.get_model(name="islamabad_aqi_model", version=None)

model_dir = Path(hw_model.download())
model = joblib.load(model_dir / "best_model.pkl")
scaler = joblib.load(model_dir / "scaler.pkl")

print(f"Model version: {hw_model.version}")
print(f"Metrics: {hw_model.training_metrics}")
print(f"Model type: {type(model).__name__}")

In [ ]:
# Cell 3: Load test data
fs = project.get_feature_store()
fg = fs.get_feature_group(name="islamabad_hourly_aqi", version=1)
df = fg.read()

df = df.sort_values("timestamp").reset_index(drop=True)
df.dropna(subset=["pm2_5"], inplace=True)

# 80/20 split - use test set for SHAP
split = int(len(df) * 0.8)
df_test = df.iloc[split:].reset_index(drop=True)

X_test = df_test[FEATURE_COLS].fillna(0)
y_test = df_test["pm2_5"].values

X_test_sc = scaler.transform(X_test)

print(f"Test set: {len(df_test)} rows")
print(f"Date range: {df_test['timestamp'].min()} → {df_test['timestamp'].max()}")

In [ ]:
# Cell 4: Create SHAP explainer
explainer = shap.TreeExplainer(model)

# Use 500 samples for speed
N = min(500, len(X_test_sc))
X_sample = X_test_sc[:N]
X_df_sample = X_test.iloc[:N]

print(f"Computing SHAP values for {N} samples...")
shap_values = explainer.shap_values(X_sample)

print(f"SHAP values shape: {shap_values.shape}")
print(f"Base value (average prediction): {explainer.expected_value:.2f} µg/m³")

In [ ]:
# Cell 5: Global feature importance (bar chart)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_df_sample, plot_type="bar", show=False)
plt.title("Global Feature Importance (Mean |SHAP| Value)", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / "shap_importance_bar.png", dpi=150, bbox_inches="tight")
plt.show()

print("✓ Saved: shap_importance_bar.png")

In [ ]:
# Cell 6: SHAP beeswarm plot (direction of impact)
plt.figure(figsize=(10, 7))
shap.summary_plot(shap_values, X_df_sample, show=False)
plt.title("SHAP Beeswarm — Feature Value Impact on PM2.5", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / "shap_beeswarm.png", dpi=150, bbox_inches="tight")
plt.show()

print("""\n✓ Saved: shap_beeswarm.png

How to read this plot:
- Each dot = one sample
- X-axis = SHAP value (impact on prediction)
- Red = high feature value, Blue = low feature value
- Red dots on the RIGHT mean: high value → higher PM2.5 predicted
""")

In [ ]:
# Cell 7: Dependence plots (top 4 features)
mean_abs_shap = np.abs(shap_values).mean(axis=0)
top4_idx = np.argsort(mean_abs_shap)[-4:][::-1]
top4_names = [FEATURE_COLS[i] for i in top4_idx]

print(f"Top 4 features: {top4_names}")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for ax, fi, fn in zip(axes.ravel(), top4_idx, top4_names):
    shap.dependence_plot(fi, shap_values, X_df_sample, ax=ax, show=False)
    ax.set_title(f"Dependence: {fn}", fontweight="bold", fontsize=12)

plt.tight_layout()
plt.savefig(ARTIFACT_DIR / "shap_dependence.png", dpi=150, bbox_inches="tight")
plt.show()

print("✓ Saved: shap_dependence.png")

In [ ]:
# Cell 8: Waterfall plot (worst-predicted hour)
preds = model.predict(X_sample)
worst_idx = int(np.argmax(preds))

exp = shap.Explanation(
    values=shap_values[worst_idx],
    base_values=explainer.expected_value,
    data=X_df_sample.iloc[worst_idx].values,
    feature_names=FEATURE_COLS,
)

shap.plots.waterfall(exp, show=False)
plt.title(f"Worst-Hour Breakdown (Predicted PM2.5 = {preds[worst_idx]:.1f} µg/m³)", fontsize=12)
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / "shap_waterfall.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Actual PM2.5: {y_test[worst_idx]:.1f} | Predicted: {preds[worst_idx]:.1f}")
print("✓ Saved: shap_waterfall.png")

In [ ]:
# Cell 9: Hourly pattern analysis
hour_idx = FEATURE_COLS.index("hour")
hour_df = pd.DataFrame({
    "hour": X_df_sample["hour"].values,
    "shap": shap_values[:, hour_idx]
})
hour_agg = hour_df.groupby("hour")["shap"].mean()

fig, ax = plt.subplots(figsize=(12, 5))
colors = ["#c1121f" if v > 0 else "#457b9d" for v in hour_agg]
ax.bar(hour_agg.index, hour_agg.values, color=colors, edgecolor="white", linewidth=0.5)
ax.axhline(0, color="black", lw=1, ls="--", alpha=0.7)
ax.set_xticks(range(0, 24))
ax.set_xlabel("Hour of Day", fontsize=12)
ax.set_ylabel("Mean SHAP Value (µg/m³ impact)", fontsize=12)
ax.set_title("How Hour-of-Day Affects PM2.5 Predictions", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.savefig(ARTIFACT_DIR / "shap_hourly.png", dpi=150, bbox_inches="tight")
plt.show()

print("✓ Saved: shap_hourly.png")

In [ ]:
# Cell 10: Feature importance summary table
importance_df = pd.DataFrame({
    "feature": FEATURE_COLS,
    "mean_abs_shap": mean_abs_shap,
    "importance_pct": (mean_abs_shap / mean_abs_shap.sum() * 100).round(2),
})
importance_df = importance_df.sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)
importance_df.to_csv(ARTIFACT_DIR / "shap_feature_importance.csv", index=False)

print("\n" + "="*60)
print("  FEATURE IMPORTANCE SUMMARY")
print("="*60)
print(importance_df.to_string(index=False))
print("="*60)

print("\n✓ Saved: shap_feature_importance.csv")
print("\nKey insights:")
print(f"- Top feature: {importance_df.iloc[0]['feature']} ({importance_df.iloc[0]['importance_pct']:.1f}%)")
print(f"- Top 3 features account for {importance_df.iloc[:3]['importance_pct'].sum():.1f}% of importance")